In [1]:
from google.colab import files
uploaded = files.upload()


Saving archive.zip to archive.zip


In [2]:
import zipfile
import os

with zipfile.ZipFile("archive.zip", 'r') as zip_ref:
    zip_ref.extractall("food101_tiny")

print(" Extracted folders:", os.listdir("food101_tiny"))


✅ Extracted folders: ['data']


In [3]:

calorie_map = {
    'apple_pie': 296,
    'burger': 295,
    'caesar_salad': 180,
    'donuts': 452,
    'edamame': 121,
    'falafel': 333,
    'french_fries': 312,
    'grilled_cheese_sandwich': 366,
    'ice_cream': 207,
    'ramen': 436
}


In [6]:
import os


print(os.listdir("food101_tiny"))


['data']


In [7]:
train_dir = "food101_tiny/data/train"
test_dir = "food101_tiny/data/test"


In [8]:
print(os.listdir("food101_tiny/data"))


['food-101-tiny']


In [9]:
train_dir = "food101_tiny/data/food-101-tiny/train"
test_dir = "food101_tiny/data/food-101-tiny/test"


In [11]:
import os

for root, dirs, files in os.walk("food101_tiny"):
    print(root)


food101_tiny
food101_tiny/data
food101_tiny/data/food-101-tiny
food101_tiny/data/food-101-tiny/valid
food101_tiny/data/food-101-tiny/valid/cannoli
food101_tiny/data/food-101-tiny/valid/apple_pie
food101_tiny/data/food-101-tiny/valid/french_toast
food101_tiny/data/food-101-tiny/valid/bibimbap
food101_tiny/data/food-101-tiny/valid/sushi
food101_tiny/data/food-101-tiny/valid/ramen
food101_tiny/data/food-101-tiny/valid/ice_cream
food101_tiny/data/food-101-tiny/valid/tiramisu
food101_tiny/data/food-101-tiny/valid/edamame
food101_tiny/data/food-101-tiny/valid/falafel
food101_tiny/data/food-101-tiny/train
food101_tiny/data/food-101-tiny/train/cannoli
food101_tiny/data/food-101-tiny/train/apple_pie
food101_tiny/data/food-101-tiny/train/french_toast
food101_tiny/data/food-101-tiny/train/bibimbap
food101_tiny/data/food-101-tiny/train/sushi
food101_tiny/data/food-101-tiny/train/ramen
food101_tiny/data/food-101-tiny/train/ice_cream
food101_tiny/data/food-101-tiny/train/tiramisu
food101_tiny/data/f

In [12]:
train_dir = "food101_tiny/data/food-101-tiny/train"
val_dir   = "food101_tiny/data/food-101-tiny/valid"


In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (224, 224)
batch_size = 16

datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_gen = datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)


Found 1500 images belonging to 10 classes.
Found 500 images belonging to 10 classes.


In [15]:
# Model building
import tensorflow as tf

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3), include_top=False, weights='imagenet'
)
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(10, activation='softmax')  # 10 classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(train_gen, validation_data=val_gen, epochs=5)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
94/94 ━━━━━━━━━━━━━━━━━━━━ 88s 866ms/step - accuracy: 0.3696 - loss: 1.8371 - val_accuracy: 0.7400 - val_loss: 0.7885
Epoch 2/5
94/94 ━━━━━━━━━━━━━━━━━━━━ 78s 836ms/step - accuracy: 0.7529 - loss: 0.7229 - val_accuracy: 0.7540 - val_loss: 0.7536
Epoch 3/5
94/94 ━━━━━━━━━━━━━━━━━━━━ 80s 852ms/step - accuracy: 0.8509 - loss: 0.4958 - val_accuracy: 0.7520 - val_loss: 0.7888
Epoch 4/5
94/94 ━━━━━━━━━━━━━━━━━━━━ 81s 860ms/step - accuracy: 0.8734 - loss: 0.4279 - val_accuracy: 0.7780 - val_loss: 0.6642
Epoch 5/5
94/94 ━━━━━━━━━━━━━━━━━━━━ 78s 829ms/step - accuracy: 0.9156 - loss: 0.2830 - val_accuracy: 0.7620 - val_loss: 0.7071


In [16]:
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array

def predict_food(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    pred = model.predict(img_array)
    idx = np.argmax(pred)
    class_name = list(train_gen.class_indices.keys())[idx]
    calories = calorie_map.get(class_name, "Unknown")

    print(f"🍽️ Predicted Food: {class_name}")
    print(f"🔥 Estimated Calories: {calories} kcal")


In [29]:
from google.colab import files
uploaded = files.upload()


Saving falafel.jpg to falafel.jpg


In [30]:
predict_food("falafel.jpg")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
🍽️ Predicted Food: falafel
🔥 Estimated Calories: 333 kcal
